<a href="https://colab.research.google.com/github/solver2005/basis_of_python/blob/main/SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#0
import sqlite3
connection = sqlite3.connect('library.db')
cursor = connection.cursor()


In [ ]:
#1
create_table_query = '''
CREATE TABLE IF NOT EXISTS books (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  title TEXT NOT NULL,
  author TEXT NOT NULL,
  year_published INTEGER,
  genre TEXT
)
'''
cursor.execute(create_table_query)
connection.commit()

In [ ]:
#2
insert_into_table = '''
  INSERT INTO books( title, author, year_published, genre)
  VALUES(?, ?, ?, ?)
  '''
books_data = [
    ("The Great Gatsby", 'F. Scott Fitzgerald', 1925, 'Fiction'),
    ("1984", 'George Orwell', 1949, 'Dystopian'),
    ("To Kill a Mockingbird", 'Harper Lee', 1960, 'Classic')
]
cursor.executemany(insert_into_table, books_data)
connection.commit()


In [ ]:
#3
select_query = '''
  SELECT title, author, genre FROM books
  '''
cursor.execute(select_query)
rows = cursor.fetchall()
for row in rows:
  print(row)


('The Great Gatsby', 'F. Scott Fitzgerald', 'Fiction')
('1984', 'George Orwell', 'Dystopian')
('To Kill a Mockingbird', 'Harper Lee', 'Classic')


In [ ]:
#4
select_query = '''
  SELECT title, author, genre FROM books WHERE year_published > ?;
  '''
required_year = 1950
cursor.execute(select_query, (required_year,))
rows = cursor.fetchall()
for row in rows:
  print(row)

('To Kill a Mockingbird', 'Harper Lee', 'Classic')


In [ ]:
#5
select_query = '''
  SELECT title, author, genre FROM books WHERE title LIKE ?;
  '''
search_pattern = 'T%'
cursor.execute(select_query, (search_pattern,))
rows = cursor.fetchall()
for row in rows:
  print(row)


('The Great Gatsby', 'F. Scott Fitzgerald', 'Fiction')
('To Kill a Mockingbird', 'Harper Lee', 'Classic')


In [ ]:
#6
select_query = '''
  SELECT title, author, genre FROM books
  ORDER BY year_published;
  '''
cursor.execute(select_query)
rows = cursor.fetchall()
for row in rows:
  print(row)

('The Great Gatsby', 'F. Scott Fitzgerald', 'Fiction')
('1984', 'George Orwell', 'Dystopian')
('To Kill a Mockingbird', 'Harper Lee', 'Classic')


In [ ]:
#7
select_query = '''
  SELECT title, author, genre FROM books
  ORDER BY title
  LIMIT 2;
  '''
cursor.execute(select_query)
rows = cursor.fetchall()
for row in rows:
  print(row)

('1984', 'George Orwell', 'Dystopian')
('The Great Gatsby', 'F. Scott Fitzgerald', 'Fiction')


In [ ]:
#8
select_query = '''
  SELECT COUNT(*) FROM books;
  '''
cursor.execute(select_query)
res = cursor.fetchone()
print(res[0])

3


In [ ]:
#9
select_query = '''
  SELECT COUNT(*) FROM books
  GROUP BY genre;
  '''
cursor.execute(select_query)
rows = cursor.fetchall()
for row in rows:
  print(row[0])

1
1
1


In [ ]:
#10
update_query = '''
UPDATE books
SET year_published = ?
WHERE title = ?;
'''
new_year = 1948
the_title = '1984'
cursor.execute(update_query, (new_year, the_title))
connection.commit()

In [ ]:
#11
delete_query = '''
DELETE FROM books
WHERE title = ?;
'''
the_title = 'The Great Gatsby'
cursor.execute(delete_query, (the_title,))
connection.commit()

In [ ]:
#12
index_query = '''
 CREATE INDEX IF NOT EXISTS idx_author ON books(author)
 '''
cursor.execute(index_query)
connection.commit()

In [ ]:
#13
cursor.execute('''
SELECT name FROM sqlite_master WHERE type='index' AND name='idx_author';
''')
result = cursor.fetchone()
if result:
  print(f"Индекс '{result[0]}' существует.")
else:
  print("Индекс не существует.")

Индекс 'idx_author' существует.


In [ ]:
#14
cursor.execute('''
CREATE TABLE IF NOT EXISTS logs(
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  book_id INTEGER,
  action TEXT,
  timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
);
''')
create_trigger_query = '''
CREATE TRIGGER IF NOT EXISTS log_books_changes
AFTER INSERT ON books
FOR EACH ROW
BEGIN
  INSERT INTO logs (book_id, action)
  VALUES (NEW.id, 'INSERT');
END;
'''
cursor.execute(create_trigger_query)
connection.commit()

In [ ]:
#15
try:
  cursor.execute('BEGIN TRANSACTION')
  cursor.execute('''
  INSERT INTO books (title, author, year_published, genre)
  VALUES(?, ?, ?, ?)
  ''', ('New_book', 'Unknown', 1920, 'Drama'))
  cursor.execute('''
  UPDATE books
  SET year_published = year_published + 1
  WHERE title = ?;
  ''', ('1984',))
  connection.commit()
  print("Транзакция успешно завершена.")
except Exception as e:
  connection.rollback()
  print(f"Ошибка: {e}. Изменения отменены.")


Транзакция успешно завершена.


In [ ]:
#16
def calculate_age(age):
  from datetime import datetime
  current_year = datetime.now().year
  return current_year - age
connection.create_function("calculate_age", 1, calculate_age)
cursor.execute('''
  SELECT calculate_age(year_published) FROM books;
''')
rows = cursor.fetchall()
for row in rows:
  print(row[0])


76
65
105


In [ ]:
#17
cursor.execute('''
CREATE VIEW IF NOT EXISTS modern_books AS
SELECT * FROM books WHERE year_published > 1950;
''')
cursor.execute('SELECT * FROM modern_books')
rows = cursor.fetchall()
for row in rows:
  print(row)


(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')


In [ ]:
#18
cursor.execute('''
DROP TABLE IF EXISTS books
''')
connection.commit()
cursor.execute('''
SELECT name FROM sqlite_master WHERE type='table' AND name='books';
''')
result = cursor.fetchone()
if result:
  print('Таблица books существует.')
else:
  print('Таблица books не существует.')

Таблица books не существует.


In [ ]:
#19
cursor.execute('''
  CREATE INDEX IF NOT EXISTS idx_author_year_published
  ON books (author, year_published);
''')
connection.commit()

In [ ]:
#20
import os
db_file = 'library.db'
connection.close()
if os.path.exists(db_file):
  print(f"Удаляем базу данных '{db_file}'...")
  os.remove(db_file)
else:
  print(f"База данных '{db_file}' не существует.")

Удаляем базу данных 'library.db'...


In [4]:
#21
import sqlite3
connection = sqlite3.connect('managment5.db')
cursor = connection.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS employee (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(255) NOT NULL,
    surname VARCHAR(255) NOT NULL,
    age INT CHECK (age >= 18), -- Возраст не может быть меньше 18
    position VARCHAR(255),
    department VARCHAR(255),
    salary INT CHECK (salary >= 0), -- Зарплата не может быть отрицательной
    isdeveloper INTEGER DEFAULT 0,
    ismanager INTEGER DEFAULT 0
);
'''
)

connection.commit()
connection.close()

In [5]:
connection = sqlite3.connect('managment5.db')
cursor = connection.cursor()
cursor.execute(
'''
INSERT INTO employee (name, surname, age, position, department, salary, isdeveloper, ismanager)
VALUES
    ('Иван', 'Иванов', 25, 'Разработчик', 'IT', 80000, 1, 0),
    ('Петр', 'Петров', 30, 'Менеджер', 'Продажи', 90000, 0, 1),
    ('Анна', 'Сидорова', 28, 'Аналитик', 'Финансы', 75000, 0, 0),
    ('Мария', 'Кузнецова', 35, 'Тестировщик', 'IT', 70000, 1, 0),
    ('Алексей', 'Смирнов', 40, 'Директор', 'Управление', 150000, 0, 1),
    ('Елена', 'Васильева', 22, 'Стажер', 'IT', 40000, 1, 0),
    ('Дмитрий', 'Федоров', 45, 'Архитектор', 'IT', 120000, 1, 0),
    ('Ольга', 'Николаева', 33, 'HR-менеджер', 'HR', 65000, 0, 1),
    ('Сергей', 'Павлов', 29, 'DevOps', 'IT', 110000, 1, 0),
    ('Татьяна', 'Михайлова', 27, 'Дизайнер', 'Маркетинг', 60000, 0, 0);
'''
)
cursor.execute('''
  SELECT * FROM employee;
''')
rows = cursor.fetchall()
for row in rows:
  print(row)
connection.close()

(1, 'Иван', 'Иванов', 25, 'Разработчик', 'IT', 80000, 1, 0)
(2, 'Петр', 'Петров', 30, 'Менеджер', 'Продажи', 90000, 0, 1)
(3, 'Анна', 'Сидорова', 28, 'Аналитик', 'Финансы', 75000, 0, 0)
(4, 'Мария', 'Кузнецова', 35, 'Тестировщик', 'IT', 70000, 1, 0)
(5, 'Алексей', 'Смирнов', 40, 'Директор', 'Управление', 150000, 0, 1)
(6, 'Елена', 'Васильева', 22, 'Стажер', 'IT', 40000, 1, 0)
(7, 'Дмитрий', 'Федоров', 45, 'Архитектор', 'IT', 120000, 1, 0)
(8, 'Ольга', 'Николаева', 33, 'HR-менеджер', 'HR', 65000, 0, 1)
(9, 'Сергей', 'Павлов', 29, 'DevOps', 'IT', 110000, 1, 0)
(10, 'Татьяна', 'Михайлова', 27, 'Дизайнер', 'Маркетинг', 60000, 0, 0)
